In [1]:
# importing libraries
from zipfile import ZipFile
import numpy as np
import openpyxl
import pandas as pd


In [2]:
def read_zip(year):
    # specifying the zip file directory
    file_name = f"/mnt/d/Mexico_Births/sinac_{year}.zip"
    
    # opening the zip file in READ mode
    with ZipFile(file_name, 'r') as zip:
        # listing all the contents of the zip file
        file_list = zip.filelist
        # getting a DataFrame from the file in the index 0
        data = pd.read_csv(zip.open(file_list[0].filename) )
    return data

In [3]:
# columns names for different year 
columns_names = {"columns_20" : ["EDAD","ESTADOCONYUGAL","ESCOLARIDAD","CLAVEOCUPACIONHABITUAL","FECHANACIMIENTO",\
"HORANACIMIENTO", "SEXO", "TALLA", "PESO", "CLUES", "ENTIDADFEDERATIVAPARTO", "MUNICIPIOPARTO",\
"LOCALIDADPARTO"], 

"columns_19" : ["EDADM","EDOCIVIL","NIV_ESCOL","CVEOCUPHAB","FECH_NACH",\
"HORA_NACH", "SEXOH", "TALLAH", "PESOH", "CLUES", "ENT_NAC", "MPO_NAC",\
"LOC_NAC"],

"columns_16" : ["edad_madre","estado_conyugal","escolaridad_madre","ocupacion_habitual_madre","fecha_nacimiento_nac_vivo",\
"hora_nacimiento_nac_vivo", "sexo_nac_vivo", "talla_nac_vivo", "peso_nac_vivo", "clues", "entidad_nacimiento", "municipio_nacimiento",\
"localidad_nacimiento"],

"new_columns": ["estado_conyugal_madre","escolaridad_madre","ocupacion_madre", "edad_madre","fecha_nacimiento_vivo",\
"hora_nacimiento_vivo", "sexo_vivo", "talla_vivo", "peso_vivo", "clues", "entidad_nacimiento", "municipio_nacimiento",\
"localidad_nacimiento"]
}

In [4]:
# reading births_colnames file which contains tables to join on main data
file_name = f"/mnt/d/Mexico_Births/births_colnames.xlsx"
aux_data = pd.read_excel(file_name, sheet_name= None)
aux_sheets = tuple(aux_data.keys())[4:] # taking sheets to join 
print(aux_sheets) 

('estado_conyugal', 'sexo', 'escolaridad', 'ocupacion2020', 'ocupacion2019')


In [5]:
aux_data[aux_sheets[0]] # estado counyugal table

,Clave_2020,Clave_2019_2016,desc_hom_cony
0,5,11,CASADA
1,1,12,SOLTERA
2,3,13,DIVORCIADA
3,2,14,VIUDA
4,4,15,UNIÓN LIBRE
5,6,16,SEPARADA
6,0,88,NO ESPECIFICADO
7,9,99,SE IGNORA


In [6]:
aux_data[aux_sheets[1]] # genero table

,Clave_2020,Clave_2019_2016,desc_hom_sex
0,0,0,NO ESPECIFICADO
1,1,1,HOMBRE
2,2,2,MUJER
3,9,9,SE IGNORA


In [7]:
aux_data[aux_sheets[2]].head() # escolaridad table

,Clave_2020,Clave_2019_2016,desc_hom_esc
0,0,0,NO ESPECIFICADO
1,1,1,NINGUNA
2,31,3,PRIMARIA COMPLETA
3,32,2,PRIMARIA INCOMPLETA
4,51,5,SECUNDARIA COMPLETA


In [12]:
def get_unified_data(year):
    data = read_zip(year) # reading zip file to get a DF

    # selecting columns
    if year == 2020:
        key_ = f"Clave_{str(year)}"
        selected_columns = columns_names[f"columns_{str(year)[-2:]}"]
        job_id = 3
    elif year in [2019, 2018, 2017]:
        selected_columns = columns_names[f"columns_19"]
        key_ = f"Clave_2019_2016"
        job_id = 4
    else:
        selected_columns = columns_names[f"columns_16"]
        key_ = f"Clave_2019_2016"
        job_id = 4
     
    data = data[selected_columns] 
    
    # joining with auxiliar tables
    data = pd.merge(data, aux_data[aux_sheets[0]], how="left", left_on=selected_columns[1], right_on=key_)
    data = pd.merge(data, aux_data[aux_sheets[1]], how="left", left_on=selected_columns[6], right_on=key_)
    data = pd.merge(data, aux_data[aux_sheets[2]], how="left", left_on=selected_columns[2], right_on=key_)
    data = pd.merge(data, aux_data[aux_sheets[job_id]], how="left", left_on=selected_columns[3], right_on=key_)

    # sorting and renaming columns
    only_columns = [-9, -3, -1, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    data = data.iloc[:,only_columns]
    data.columns = columns_names["new_columns"]

    return data

In [13]:
data = get_unified_data(2019)

/tmp/ipykernel_22401/3537065925.py:2: DtypeWarning: Columns (7,51,56,63,65) have mixed types.Specify dtype option on import or set low_memory=False.
  data = read_zip(year) # reading zip file to get a DF
/tmp/ipykernel_22401/3537065925.py:24: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Clave_2019_2016_x'} in the result is deprecated and will raise a MergeError in a future version.
  data = pd.merge(data, aux_data[aux_sheets[job_id]], how="left", left_on=selected_columns[3], right_on=key_)


In [14]:
data.sample(3)

,estado_conyugal_madre,escolaridad_madre,ocupacion_madre,edad_madre,fecha_nacimiento_vivo,hora_nacimiento_vivo,sexo_vivo,talla_vivo,peso_vivo,clues,entidad_nacimiento,municipio_nacimiento,localidad_nacimiento
1197261,CASADA,SECUNDARIA COMPLETA,NO OCUPADO,36,22/12/2019,14:02:00,2,52,3700,NLIMS000636,19,6,1
33588,UNIÓN LIBRE,PRIMARIA COMPLETA,NO REMUNERADO - AMA DE CASA,34,05/01/2019,20:40:00,1,50,3690,BCSSA000015,2,1,1
173089,UNIÓN LIBRE,LICENCIATURA O PROFESIONAL INCOMPLETO,NO REMUNERADO - AMA DE CASA,23,17/01/2019,04:17:00,1,49,3570,CSSSA009244,7,89,1


In [6]:
data = data[columns_20]

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747847 entries, 0 to 1747846
Data columns (total 13 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   EDAD                    int64 
 1   ESTADOCONYUGAL          int64 
 2   ESCOLARIDAD             int64 
 3   CLAVEOCUPACIONHABITUAL  int64 
 4   FECHANACIMIENTO         object
 5   HORANACIMIENTO          object
 6   SEXO                    int64 
 7   TALLA                   int64 
 8   PESO                    int64 
 9   CLUES                   object
 10  ENTIDADFEDERATIVAPARTO  int64 
 11  MUNICIPIOPARTO          int64 
 12  LOCALIDADPARTO          int64 
dtypes: int64(10), object(3)
memory usage: 173.4+ MB


In [8]:
data = pd.merge(data, aux_data[aux_sheets[0]], how="left", left_on="ESTADOCONYUGAL", right_on="Clave_2020")
data = pd.merge(data, aux_data[aux_sheets[1]], how="left", left_on="SEXO", right_on="Clave_2020")
data = pd.merge(data, aux_data[aux_sheets[2]], how="left", left_on="ESCOLARIDAD", right_on="Clave_2020")
data = pd.merge(data, aux_data[aux_sheets[3]], how="left", left_on="CLAVEOCUPACIONHABITUAL", right_on="Clave_2020")

/tmp/ipykernel_15788/3021495784.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Clave_2020_x'} in the result is deprecated and will raise a MergeError in a future version.
  data = pd.merge(data, aux_data[aux_sheets[3]], how="left", left_on="CLAVEOCUPACIONHABITUAL", right_on="Clave_2020")


In [16]:
data.columns

Index(['EDAD', 'ESTADOCONYUGAL', 'ESCOLARIDAD', 'CLAVEOCUPACIONHABITUAL',
       'FECHANACIMIENTO', 'HORANACIMIENTO', 'SEXO', 'TALLA', 'PESO', 'CLUES',
       'ENTIDADFEDERATIVAPARTO', 'MUNICIPIOPARTO', 'LOCALIDADPARTO',
       'Clave_2020_x', 'Clave_2019_2015_x', 'desc_hom_cony', 'Clave_2020_y',
       'Clave_2019_2015_y', 'desc_hom_sex', 'Clave_2020_x', 'Clave_2019_2015',
       'desc_hom_esc', 'Clave_2020_y', 'desc_hom_ocup'],
      dtype='object')

In [13]:
only_columns = [-9, -3, -1, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [17]:
data = data.iloc[:,only_columns]

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1747847 entries, 0 to 1747846
Data columns (total 13 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   desc_hom_cony           object
 1   desc_hom_esc            object
 2   desc_hom_ocup           object
 3   EDAD                    int64 
 4   FECHANACIMIENTO         object
 5   HORANACIMIENTO          object
 6   SEXO                    int64 
 7   TALLA                   int64 
 8   PESO                    int64 
 9   CLUES                   object
 10  ENTIDADFEDERATIVAPARTO  int64 
 11  MUNICIPIOPARTO          int64 
 12  LOCALIDADPARTO          int64 
dtypes: int64(7), object(6)
memory usage: 186.7+ MB


In [22]:
data.columns = ["estado_conyugal_madre","escolaridad_madre","ocupacion_madre", "edad_madre","fecha_nacimiento_vivo",\
    "hora_nacimiento_vivo", "sexo_vivo", "talla_vivo", "peso_vivo", "clues", "entidad_nacimiento", "municipio_nacimiento",\
"localidad_nacimiento"]

In [35]:
data.sample(3)

,estado_conyugal_madre,escolaridad_madre,ocupacion_madre,edad_madre,fecha_nacimiento_vivo,hora_nacimiento_vivo,sexo_vivo,talla_vivo,peso_vivo,clues,entidad_nacimiento,municipio_nacimiento,localidad_nacimiento
1383044,UNIÓN LIBRE,BACHILLERATO O PREPARATORIA COMPLETA,"OTROS OPERADORES DE MAQUINARIA INDUSTRIAL, ENS...",27,08/06/2020,23:40,1,54,3400,DGIMS000042,10,5,1
336313,CASADA,LICENCIATURA O PROFESIONAL COMPLETO,"TRABAJADORES DOMÉSTICOS, DE LIMPIEZA, PLANCHAD...",28,04/05/2020,06:58,1,48,2535,SLSMP001024,25,1,70
1672466,SOLTERA,SECUNDARIA COMPLETA,"NO REMUNERADO, AMA DE CASA",21,19/05/2020,13:22,2,48,2990,JCSSA002212,14,39,1
